- 참고 : https://jeongwookie.github.io/2019/06/10/190610-twitter-data-crawling/

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#%cd /content/drive/My Drive

/content/drive/My Drive/data


In [0]:
#1월 29일

In [1]:
# GetOldTweet3 사용 준비
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# BeautifulSoup4 사용 준비
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup

In [5]:
# 가져올 범위를 정의
# 예제 : 2020-01-29 ~ 2020-01-29

import datetime

days_range = []

start = datetime.datetime.strptime("2020-01-29", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-01-30", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))



=== 설정된 트윗 수집 기간은 2020-01-29 에서 2020-01-29 까지 입니다 ===
=== 총 1일 간의 데이터 수집 ===


In [6]:
# 특정 검색어가 포함된 트윗 검색하기 (query search)


import time

# 수집 기간 맞추기
start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('(신종 +코로나) OR (우한 +폐렴)')\
                                           .setSince(start_date)\
                                           .setUntil(end_date)\
                                           .setMaxTweets(-1)


#tweetCriteria = got.manager.TweetCriteria().setQuerySearch('(신종 +코로나) OR (우한 +폐렴) OR (신종코로나) OR (우한폐렴)')\
#                                           .setSince(start_date)\
#                                           .setUntil(end_date)\
#                                           .setMaxTweets(-1)

# 수집 with GetOldTweet3
print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
start_time = time.time()

tweet = got.manager.TweetManager.getTweets(tweetCriteria)

print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
print("=== Total num of tweets is {} ===".format(len(tweet)))


=== Total num of tweets is 5092 ===


In [0]:
# 원하는 변수 골라서 저장하기

from random import uniform
from tqdm import tqdm_notebook

# initialize
tweet_list = []

for index in tqdm_notebook(tweet):
    
    # 메타데이터 목록 
    username = index.username
    link = index.permalink 
    content = index.text
    tweet_date = index.date.strftime("%Y-%m-%d")
    tweet_time = index.date.strftime("%H:%M:%S")
    retweets = index.retweets
    favorites = index.favorites
    
    # === 유저 정보 수집 시작 ===
    #try:
        #personal_link = 'https://twitter.com/' + username
        #bs_obj = get_bs_obj(personal_link)
        #uls = bs_obj.find("ul", {"class": "ProfileNav-list"}).find_all("li")
        #div = bs_obj.find("div", {"class": "ProfileHeaderCard-joinDate"}).find_all("span")[1]["title"]


        # 가입일, 전체 트윗 수, 팔로잉 수, 팔로워 수
        #joined_date = div.split('-')[1].strip()
        #num_tweets = uls[0].find("span", {"class": "ProfileNav-value"}).text.strip()
        #num_following = uls[1].find("span", {"class": "ProfileNav-value"}).text.strip()
        #num_follower = uls[2].find("span", {"class": "ProfileNav-value"}).text.strip()
    
    #except AttributeError:
        #print("=== Attribute error occurs at {} ===".format(link))
        #print("link : {}".format(personal_link))   
        #pass
     
    # 결과 합치기
    info_list = [tweet_date, tweet_time, username, content, link, retweets, favorites]
    tweet_list.append(info_list)
    
    # 휴식 
    time.sleep(uniform(1,2))

In [0]:
import pandas as pd

twitter_df = pd.DataFrame(tweet_list, 
                          columns = ["date", "time", "user_name", "text", "link", "retweet_counts", "favorite_counts"])

# csv 파일 만들기
twitter_df.to_csv("sample_twitter_data_{}_to_{}.csv".format(days_range[0], days_range[-1]), index=False)
print("=== {} tweets are successfully saved ===".format(len(tweet_list)))